# Custom Question Answering Solution

## Install Library

In [ ]:
%pip install azure-ai-language-questionanswering

## Load Azure Configurations

In [6]:
import os

# Load Azure configurations from environment variables
# Ensure that AZURE_AI_LANGUAGE_KEY and AZURE_AI_LANGUAGE_ENDPOINT are set in your environment
language_key = os.environ.get('AZURE_AI_LANGUAGE_KEY')
language_endpoint = os.environ.get('AZURE_AI_LANGUAGE_ENDPOINT')

# The name of your question answering project in Language Studio
knowledge_base_project = "AzureServices"

# There are two possible values: test, and production. 
# production is dependent on you having deployed your project from Language Studio > question answering > Deploy project.
deployment = "test"

## Create a Question Answering Client

In [2]:
from azure.ai.language.questionanswering import QuestionAnsweringClient
from azure.ai.language.questionanswering.models import ShortAnswerOptions
from azure.core.credentials import AzureKeyCredential

# Authenticate the client using Azure Key and Endpoint
def authenticate_client():
    """
    Authenticate the Question Answering client using Azure Key Credential.

    Returns:
        QuestionAnsweringClient: Authenticated client for Question Answering.
    """
    qac_credential = AzureKeyCredential(language_key)
    question_answering_client = QuestionAnsweringClient(
        language_endpoint, 
        qac_credential)
    
    return question_answering_client

# Initialize the client
client = authenticate_client()

## Get Answers Function

In [13]:
def get_answers_with_project(client, question, knowledge_base_project, deployment):
    """
    Retrieves answers from the Azure Custom Question Answering service for a given question.

    Args:
        client (QuestionAnsweringClient): The authenticated Question Answering client.
        question (str): The question to query the knowledge base.
        knowledge_base_project (str): The name of the knowledge base project in Azure Language Studio.
        deployment (str): The deployment environment (e.g., 'test' or 'production').

    Returns:
        None: Prints the answers and their details to the console.
    """
    output = client.get_answers(
        project_name=knowledge_base_project,
        deployment_name=deployment,
        question=question,
        confidence_threshold=0.10,
        top=5,
        short_answer_options=ShortAnswerOptions(confidence_threshold=0.7, top=1),
    )

    print(f"Q: {question}")
    for idx, answer in enumerate(output.answers):
        print(f"Answer {idx + 1}:")
        print(f"Alternate questions: {answer.questions}")
        print(f"Answer Text: {answer.answer}")
        print(f"Confidence: {answer.confidence:.2f}")
        print(f"Source: {answer.source}")

        # Check if a short answer is available and print its details
        if answer.short_answer:
            short_answer = answer.short_answer
            print("Short Answer:")
            print(f"  Text: {short_answer.text.strip()}")
            print(f"  Confidence: {short_answer.confidence:.2f}")
            print(f"  Offset: {short_answer.offset}")
            print(f"  Length: {short_answer.length}")

        # Display follow-up prompts if available
        if answer.dialog and answer.dialog.prompts:
            print("Follow-up Prompts:")
            for prompt in answer.dialog.prompts:
                print(f"  - {prompt.display_text}")  # Display text of the follow-up prompt
        print("-" * 50)  # Separator for readability

## Display Answers

In [14]:
question = "What is Azure SQL?"
get_answers_with_project(client, question, knowledge_base_project, deployment)

Q: What is Azure SQL?
Answer 1:
Alternate questions: ['Azure SQL Database']
Answer Text:  Azure SQL Database is a fully managed relational database service based on the latest stable version of Microsoft SQL Server. It offers built-in intelligence that learns your application patterns and adapts to maximize performance, reliability, and data protection. SQL Database supports elastic scaling, allowing you to dynamically adjust resources to match your workload. It provides advanced security features, such as encryption, auditing, and threat detection. You can migrate your existing SQL Server databases to Azure SQL Database with minimal downtime. 
Confidence: 0.62
Source: azure_services.docx
Follow-up Prompts:
  - Azure Database for PostgreSQL
  - Azure Database for MariaDB
--------------------------------------------------
Answer 2:
Alternate questions: ['Azure SQL Managed Instance']
Answer Text:  Azure SQL Managed Instance is a fully managed, scalable, and secure SQL Server instance hos